# This notebook generates the pickle files based on the `.rda` files in the original repository
## Doing this in a notebook for the sake of convenience and because it only has to be done once. 

In [1]:
import numpy as np
import pandas as pd
import warnings
import pyreadr
from importlib import reload
import wrangle as w


In [ ]:
warnings.simplefilter("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [ ]:
contestants = pyreadr.read_r(
    path='data/rpdr_contestants.rda')['rpdr_contestants']
contestants.season = contestants.season.str[1:].astype('int')
contestants.age = contestants.age.astype('int')
contestants.dob = pd.to_datetime(contestants.dob)
hometown_df = contestants.hometown.str.split(', ', expand=True)
hometown_df = hometown_df.rename(
    columns={0: 'city', 1: 'state'}).drop(columns=[2])
hometown_df.city = hometown_df.city.astype('category')
hometown_df.state = hometown_df.state.astype('category')
contestants = pd.concat([contestants, hometown_df],
                        axis=1).drop(columns=['hometown'])
contestants.loc[contestants.contestant == 'Jaida Essence Hall'].index = [144]
contestants.loc[contestants.contestant == 'Crystal Methyd'].index = [146]
crystal = contestants.loc[144].copy(deep=True)
contestants.loc[144] = contestants.loc[146]
contestants.loc[146] = crystal
contestants = contestants.rename(columns={'contestant': 'queen_name'})
winners = contestants.groupby('season').first().queen_name.to_list()
contestants['winner'] = contestants.queen_name.isin(winners)
contestants.head()


In [ ]:
def get_episode_type(row: pd.Series) -> str:
    if row.finale > 0:
        return 'Finale'
    elif row.penultimate > 0:
        return 'Penultimate'
    else:
        return 'Standard'


def check_elimination(row: pd.Series) -> pd.Series:
    if row.eliminated > '0':
        row.outcome = 'ELIM'
    return row

contep = pyreadr.read_r('data/rpdr_contep.rda')['rpdr_contep']
contep = contep.dropna(subset=['outcome'])
contep.season = contep.season.str[1:].astype(np.int8)
contep.episode = contep.episode.astype(np.int8)
contep['rank'] = contep['rank'].fillna(-1).astype(np.int8)
contep['rank'].name = 'ranking'
contep = contep.apply(check_elimination, axis=1)
contep['etype'] = contep.apply(get_episode_type, axis=1)
contep.etype = contep.etype.astype('category')

contep = contep.drop(
    columns=['penultimate', 'finale', 'participant', 'eliminated'])
for index, name in contestants.queen_name.to_dict().items():
    contep.loc[contep.contestant == name, 'contestant'] = index
contep.minichalw = contep.minichalw.apply(
    lambda w: True if w == '1' else False)
contep.missc = contep.missc.apply(lambda w: True if w == '1' else False)
contep = contep.rename(
    columns={'contestant': 'queen_id', 'missc': 'ms_congeniality'})
contep.to_pickle('data/contep.pkl')
contestants.to_pickle('data/contestants.pkl')


In [ ]:
def get_episode_type(row: pd.Series) -> str:
    if row.special > 0:
        return 'Special'
    elif row.finale > 0:
        return 'Finale'
    else:
        return 'Standard'


episodes = pyreadr.read_r('data/rpdr_ep.rda')['rpdr_ep']
episodes.season = episodes.season.str[1:]
episodes.minic = episodes.minic.fillna('None')
episodes.numqueens = episodes.numqueens.fillna(0)
episodes.runwaytheme = episodes.runwaytheme.fillna('Not Listed')

bottom_drop = ['bottom'+str(i) for i in range(1, 8)]
minicw_drop = ['minicw'+str(i) for i in range(1, 5)]
episodes = episodes.drop(columns=[
                         'special', 'finale', 'eliminated1', 'eliminated2'] + bottom_drop + minicw_drop)
# episodes.to_csv('data/episodes.csv')


In [ ]:
episodes = pd.read_pickle()
episodes.season = episodes.season.astype(np.uint8)
episodes.episode = episodes.episode.astype(np.uint8)
episodes.airdate = pd.to_datetime(episodes.airdate)
episodes.numqueens = episodes.numqueens.astype(np.uint16)
episodes.main_challenge = episodes.main_challenge.astype('category')
episodes = episodes.rename(columns={'airdate': 'air_date', 'runwaytheme': 'runway_theme', 'numqueens': 'n_queens',
                           'minic': 'mini_challenge', 'lipsyncartist': 'lipsync_artist', 'lipsyncsong': 'lipsync_song'})
episodes.to_pickle('data/episodes.pkl')


FileNotFoundError: [Errno 2] No such file or directory: 'data/episodes.csv'

## Reducing the number of outcomes
The original dataset had a number of one-off or situational outcomes. This section was used to build the code to clean these outcomes

In [8]:
reload(w)
queens, contep, episodes = w.acquire_rpdr_data()
train, validate, test = w.split_queens((queens, contep, episodes))
df = pd.concat([train,validate,test]).sort_values(['season','episode','rank'],ascending=[True,True,False])
df.outcome.value_counts()


SAFE     459
HIGH     247
WIN      185
BTM      176
ELIM     144
LOW      122
GUEST     61
WDR        2
Name: outcome, dtype: int64

In [173]:
bottoms = df[df.outcome == 'BTM'][['season','episode','nickname','queen_name','outcome','rank']].sort_values(by=['season','rank'],ascending=[True, False])
elim = df[df.outcome == 'ELIM'][['season','episode','nickname','queen_name','outcome','rank']].sort_values(by=['season','rank'],ascending=[True, False])

In [12]:
df[df.queen_name.str.contains('Hara')]

,season,queen_name,age,dob,city,state,winner,rank,ms_congeniality,episode,...,minichalw,etype,air_date,nickname,runway_theme,n_queens,mini_challenge,lipsync_artist,lipsync_song,challenge_type
251,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,1,...,False,Standard,2012-01-30,RuPocalypse Now!,Not Listed,13,Photoshoot with Mike Ruiz,Britney Spears,Toxic,Fabrication
260,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,2,...,False,Standard,2012-02-06,WTF!: Wrestling's Trashiest Fighters,Girly Girl Couture,12,Create “badonkadonk” with padding in 30 minutes,Donna Summer,Bad Girls,Performance
269,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,3,...,False,Standard,2012-02-13,Glamazons vs Champions,Platinum and Gold,11,Create hats with butterfly-shaped accessories,Natalie Cole,This Will Be (An Everlasting Love),Performance
278,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,4,...,False,Standard,2012-02-20,Queens Behind Bars,Not Listed,10,"Paint a partner's face, while handcuffed ...",Pink,Trouble,Performance
287,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,5,...,True,Standard,2012-02-27,Snatch Game,Best Drag,9,Beat the Clock,Madonna,Vogue,Snatch Game
413,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,6,...,False,Standard,2012-03-05,Float Your Boat,Not Listed,8,Wet t-shirt contest,Lady Gaga,Born This Way,Fabrication
532,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,7,...,False,Standard,2012-03-12,Dragazines,Not Listed,7,Reading is Fundamental,Pam Tillis,Mi Vida Loca,Comedy
538,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,8,...,False,Standard,2012-03-19,Frenemies,Not Listed,6,Polygraph test,Martha Wash and RuPaul,It's Raining Men (The Sequel),Performance
543,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,9,...,True,Standard,2012-03-26,Frock the Vote!,Inaugural Drag,5,Design Absolut Vodka-inspired shoe,Gladys Knight,I've Got to Use My Imagination,Performance
294,4,Phi Phi O'Hara,25,1985-10-10,Chicago,Illinois,False,2,False,10,...,False,Standard,2012-04-02,DILFs: Dads I'd Like To Frock,Not Listed,5,Give a teddy bear a makeover,Aretha Franklin,(You Make Me Feel Like) A Natural Woman,Makeover


Data is missing all mini challenge wins after season 10, so we're gonna fill those in (ugh)

In [4]:
from bs4 import BeautifulSoup
import re
import requests
response = requests.get('https://en.wikipedia.org/wiki/RuPaul%27s_Drag_Race_(season_7)').text
response = re.sub(r'<(/)?b>','',response)
bs = BeautifulSoup(response,'html.parser')
table = bs.find('table',class_='wikiepisodetable').find_all('td')
episodes = []
items_to_get = [('mini-challenge','mini_challenge'),('mini-challenge winner','mini_challenge_winner'),('mini-challenge winners','mini_challenge_winner'),('main challenge','main_challenge'),('runway theme','runway_theme'),('guest judges','guest_judges'),('guest judge','guest_judge')]
for i in range(0,len(table),4):
    episode = {'season':7}
    episode['episode'] = int(table[i].get_text())
    episode['episode_name'] = table[i+1].get_text()
    episode['air_date'] = table[i+2].find('span',class_='bday').get_text()
    description_lists = table[i+3].find_all('ul')
    if len(description_lists) > 0:
        ep_info = description_lists[len(description_lists)-1].find_all('li')
        info_items = {e.get_text().split(': ')[0].lower():e.get_text().split(':')[1] for e in ep_info}
        for item in items_to_get:
            if item[0] in info_items.keys():
                episode[item[1]] = info_items[item[0]]

    episodes.append(pd.Series(episode))
pd.DataFrame(episodes)



,season,episode,episode_name,air_date,mini_challenge,runway_theme,guest_judge,mini_challenge_winner,guest_judges
0,7,1,"""Born Naked""",2015-03-02,Walk the runway showing off two looks; one fr...,Nude Illusion,Kathy Griffin,NaN,NaN
1,7,2,"""Glamazonian Airways""",2015-03-09,Give face while the pit-crew blows them with ...,Jet Set Eleganza,NaN,Ginger Minj and Trixie Mattel,Jordin Sparks and Olivia Newton-John
2,7,3,"""ShakesQueer""",2015-03-16,"The queens dress up in their finest ""granny d...",Bearded and Beautiful,NaN,Kennedy Davenport and Max,Mel B and Kat Dennings
3,7,4,"""Spoof! (There It Is)""",2015-03-23,NaN,Green,NaN,NaN,Jessica Alba and Lucian Piane
4,7,5,"""The DESPY Awards""",2015-03-30,Make their own paper versions of classic cele...,NaN,NaN,Katya and Mrs. Kasha Davis,Isaac Mizrahi
5,7,6,"""RuPaullywood Stories""",2015-04-06,Gain points by selecting pit crew members whi...,Death Becomes Her,NaN,Ginger Minj,Ariana Grande and Merle Ginsberg
6,7,7,"""Snatch Game""",2015-04-13,NaN,Leather and Lace,Michael Urie and Tamar Braxton[17],NaN,NaN
7,7,8,"""Conjoined Queens""",2015-04-20,"Recreate prison uniforms as ""Orange is the ne...",NaN,LeAnn Rimes and Nelsan Ellis[18],Kennedy Davenport,NaN
8,7,9,"""Divine Inspiration""",2015-04-27,Reading is Fundamental,Ugliest Dress,Demi Lovato and John Waters,Trixie Mattel,NaN
9,7,10,"""Prancing Queens""",2015-05-04,"Create a ""Fake Housewives of Drag Race"" look ...","Half Man, Half Queen",Alyssa Milano and Rachael Harris,Violet Chachki,NaN


In [79]:
import time
from importlib import reload
import wrangle as w
reload(w)
season_seven = w.scrape_episode_info('https://en.wikipedia.org/wiki/RuPaul%27s_Drag_Race_(season_{:d})',7)
pd.DataFrame(season_seven)

,season,episode,episode_name,air_date,mini_challenge,runway_theme,mini_challenge_winner
0,7,1,"""Born Naked""",2015-03-02,Walk the runway showing off two looks; one fro...,Nude Illusion,NaN
1,7,2,"""Glamazonian Airways""",2015-03-09,Give face while the pit-crew blows them with a...,Jet Set Eleganza,Ginger Minj and Trixie Mattel
2,7,3,"""ShakesQueer""",2015-03-16,"The queens dress up in their finest ""granny dr...",Bearded and Beautiful,Kennedy Davenport and Max
3,7,4,"""Spoof! (There It Is)""",2015-03-23,NaN,Green,NaN
4,7,5,"""The DESPY Awards""",2015-03-30,Make their own paper versions of classic celeb...,NaN,Katya and Mrs. Kasha Davis
5,7,6,"""RuPaullywood Stories""",2015-04-06,Gain points by selecting pit crew members whil...,Death Becomes Her,Ginger Minj
6,7,7,"""Snatch Game""",2015-04-13,NaN,Leather and Lace,NaN
7,7,8,"""Conjoined Queens""",2015-04-20,"Recreate prison uniforms as ""Orange is the new...",NaN,Kennedy Davenport
8,7,9,"""Divine Inspiration""",2015-04-27,Reading is Fundamental,Ugliest Dress,Trixie Mattel
9,7,10,"""Prancing Queens""",2015-05-04,"Create a ""Fake Housewives of Drag Race"" look u...","Half Man, Half Queen",Violet Chachki


In [45]:

episodes = []
for i in range(1,15):
    print(f'Season {i}')
    episodes += w.scrape_episode_info(f'https://en.wikipedia.org/wiki/RuPaul%27s_Drag_Race_(season_{i})',i)
    time.sleep(10)
episodes = pd.DataFrame(episodes)


Season 1
Season 2
Season 3
Season 4
Season 5
Season 6
Season 7
Season 8
Season 9
Season 10
Season 11
Season 12
Season 13
Season 14


,season,episode,episode_name,air_date,mini_challenge,main_challenge,mini_challenge_winner,runway_theme,guest_judge
0,1,1,"""Drag on a Dime""",2009-02-02,Sexy car wash photo shoot,Making an outfit out of items from the thrift...,NaN,NaN,NaN
1,1,2,"""Girl Groups""",2009-02-09,Act out certain emotions,Destiny's Child girl group battle,Akashia and Ongina,NaN,NaN
2,1,3,"""Queens of All Media""",2009-02-16,NaN,Channeling Oprah in three stages of her caree...,NaN,NaN,NaN
3,1,4,"""Mac-Viva Glam""",2009-02-23,Give your partner a 30 minute makeover,Mac-Viva Glam commercial,Jade,NaN,NaN
4,1,5,"""Drag School of Charm""",2009-03-02,Outlast your opponents in an endurance challenge,Transforming a female fighter in their own image,Rebecca Glasscock,NaN,NaN
...,...,...,...,...,...,...,...,...,...
186,14,12,"""Moulin Ru: The Rusical""",2022-03-25,NaN,Moulin Ru: The Rusical,NaN,": Mirror, Mirror",NaN
187,14,13,"""The Ross Mathews Roast""",2022-04-01,"In teams, paint a mural of Dolly Parton or RuPaul",A roast of Ross Mathews,"Bosco, Daya Betty, Lady Camden and Willow Pill",: Tutu Much,NaN
188,14,14,"""Catwalk""",2022-04-08,NaN,"Write, record and perform their own verses to...",NaN,: You're A Winner Baby,NaN
189,14,15,"""Reunited""",2022-04-15,NaN,NaN,NaN,NaN,NaN


In [75]:
i = 7
response = requests.get('https://en.wikipedia.org/wiki/RuPaul%27s_Drag_Race_(season_7)').text

bs = BeautifulSoup(response,'html.parser')
li = bs.find('table', class_='wikiepisodetable').find_all('li')[13]
li.b.decompose()
li

<li>: Give face while the pit-crew blows them with a leaf blower with special guest <a href="/wiki/Moby" title="Moby">Moby</a>.</li>